# Image Recoloring
**TODO:**


* Dataset class
* Training and test split
* Data augmentation (?)
* Model of generator and discriminator
* Loss function
* Traing procedure



## Setup




In [1]:
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import time as time
import numpy as np
from IPython import display

## Prepare dataset

The dataset is shared here https://drive.google.com/file/d/1Zq46n_VFuENm1OLaRaKQkIqq8wA__vFq/view?usp=sharing. Add a shortcut to your own google drive and mount drive on google colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/COCOtrain2014.zip
!rm -rf /content/__MACOSX

ALby:


*   Classe dataset e dataloader
*   Resize a 256 x 256
* Bianco e nero 
* split train e test (80:20)






## Create the models

### Generator

In [ ]:
# codice

### Discriminator
The architecture for the PatchGAN Discriminator (70x70 patch) is:
 

1.   Convolutional layer with 64 filters, kernel size=4, stride=2, padding=1, LeakyRelu (with negative_slope=0.2)
2.   Convolutional layer with 128 filters, kernel size=4 stride=2, padding=1, LeakyRelu (with negative_slope=0.2) and with Batch Normalization
3.   Convolutional layer with 256 filters, kernel size=4 stride=2, padding=1, LeakyRelu (with negative_slope=0.2) and with Batch Normalization
4.   Convolutional layer with 512 filters, kernel size=4 stride=1, padding=1, LeakyRelu (with negative_slope=0.2) and with Batch Normalization


In [28]:
class CNNBlockBN(nn.Module):
  def __init__(self, in_channels, out_channels, s=2):

    super().__init__()
    
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 4, stride=s, bias=False, padding_mode="reflect"),
        nn.BatchNorm2d(out_channels),
        nn.LeakyReLU(0.2),
    )
  
  def forward(self, x):
    return self.conv(x)

# CNN without Batch Normalization
class CNNBlock(nn.Module):
  def __init__(self, in_channels, out_channels, s=2):

    super().__init__()
    
    self.conv = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 4, stride=s, padding=1, padding_mode="reflect"),
        nn.LeakyReLU(0.2),
    )
  
  def forward(self, x):
    return self.conv(x)

class Discriminator(nn.Module):
  def __init__(self):
    super().__init__()
    features = [64, 128, 256, 512]

    self.model = nn.Sequential(
        CNNBlock(3*2, features[0]),    #*2 two image concatenated 
        CNNBlockBN(features[0], features[1]),
        CNNBlockBN(features[1], features[2]),
        # last block with stride = 1
        CNNBlockBN(features[2], features[3], s=1),
        nn.Conv2d(features[3], 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect")
    )
  
  def forward(self, x, y):
    x = torch.cat([x, y], dim=1)
    x = self.model(x)
    return x

In [29]:
# test the model 

def test():
  # 1 image with 3 channels 256x256
  x = torch.randn((1, 3, 256, 256))
  y = torch.randn((1, 3, 256, 256))
  model = Discriminator()
  preds = model(x, y)
  print(model)
  print(preds.shape)

test()

Discriminator(
  (model): Sequential(
    (0): CNNBlock(
      (conv): Sequential(
        (0): Conv2d(6, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), padding_mode=reflect)
        (1): LeakyReLU(negative_slope=0.2)
      )
    )
    (1): CNNBlockBN(
      (conv): Sequential(
        (0): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), bias=False, padding_mode=reflect)
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
    (2): CNNBlockBN(
      (conv): Sequential(
        (0): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), bias=False, padding_mode=reflect)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
    (3): CNNBlockBN(
      (conv): Sequential(
        (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), bias=False, padding_mode=reflect)
        (1): BatchNor

## Define the loss

### Generator loss

In [ ]:
# codice

### Discriminator loss

In [ ]:
# codice

## Training

In [ ]:
# codice